<a href="https://colab.research.google.com/github/Aninditadas2307/Advanced-House-Price-Prediction-/blob/master/Copy_of_Latest_Code_Final_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io
import cv2

In [3]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

--2020-11-26 16:23:49--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml.1’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.04s   

2020-11-26 16:23:49 (22.7 MB/s) - ‘haarcascade_frontalface_default.xml.1’ saved [930127/930127]



In [5]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

In [6]:
!apt update
!apt install -y cmake
!pip install dlib
!apt install unixodbc-dev
!pip install pyodbc


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

In [7]:
!pip install face-recognition

     |████████████████████████████████| 100.2MB 108kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=d0bc830ab1d411d54359556b86fed4af5a17d2e4ba3012d15c90b16f3966fa98
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [8]:
!pip install pyttsx3

In [9]:
import cv2
import os
import face_recognition
import numpy as np
import pandas as pd
from scipy import stats
import itertools
import pyttsx3

In [10]:
from google.colab import files
files.upload()

Saving encoding.csv to encoding.csv


{'encoding.csv': b'"Darshan,Vivek,harish,Sathish,Anindita,Abhiroop,Prashantha,Debyendu Dey,Harish,Vivek Muraleedharan,James Ajeeth"\r\n"-0.141976312,-0.120766468,-0.146423697,-0.125963748,-0.067408405,-0.15050754,-0.128416166,-0.15962742300000002,-0.088019304,-0.1017017588019371,-0.10024460405111313"\r\n"0.00258876,0.098219551,0.088945501,-0.019857496,0.016406815,0.138736129,0.059278779,0.12834017,0.047893405,0.0622192919254303,0.05932195112109184"\r\n"-0.009749264,-0.006743179,0.014782903,0.040006123,0.027709581,0.043995377,0.103477091,-0.001202059,0.06059926700000001,0.02403281070291996,0.10969705879688263"\r\n"-0.066246808,-0.075809129,-0.039824791,-0.007705198000000002,-0.029159149,0.004728511,-0.023915263,-0.066211,-0.037857156,-0.027431588619947437,0.02591830864548683"\r\n"-0.06980764099999999,-0.035339057,0.0300634,-0.072141312,-0.13396370400000002,0.008314165,-0.113412693,-0.083098255,0.030682752,-0.009559722617268562,-0.06981310248374939"\r\n"-0.016149471999999998,-0.012935214

In [11]:
!sudo apt-get install espeak

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 62 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak amd64 1.48.04+dfsg-5 [61.6 kB]
Fetched 1,219 kB in 1s (850 kB/s)
deb

In [12]:
faces=[]
face_known_encodings=[]
face_known_name = []
pred_names = []
face_known_encodings_flat=[]
engine = pyttsx3.init()

In [13]:
def video2encoding(vid_source,names):
    cam = cv2.VideoCapture('/content/Satish_Video.mp4')
    frame_no = 0
    know_encodings = []

    while True: 

        # reading from frame 
        ret,frame = cam.read()
        if len(know_encodings) == 45:
            break
        #print(ret)
        if ret==True:
            #print (frame_no)
            if frame_no%6 == 1:
                face_locations = face_recognition.face_locations(frame)
                if len(face_locations)!= 0:
                    face_encoding = face_recognition.face_encodings(frame,known_face_locations=face_locations)
                    know_encodings.append(face_encoding)
            frame_no = frame_no+1
            cv2.imshow('img', frame)
            cv2.waitKey(1)
        else:
            break

    # Release all space and windows once done
    
    cam.release() 
    cv2.destroyAllWindows() 
    faces.append([names,know_encodings])
    
    print("done")

In [14]:
df= pd.DataFrame()
def encoding_df (faces):
   
    for j in range(0,len(faces)):
        test = faces[j][1]
        name = faces[j][0]
        #print(name)
        a=list(itertools.chain(*test))
        b=list(itertools.chain(*a))
        df[[name]]=pd.DataFrame(b)

    return(df)


In [15]:
def add_new_encoding(source,faces):
    df=pd.read_csv(source)
    for j in range(0,len(faces)):
        test = faces[j][1]
        name = faces[j][0]
        #print(name)
        a=list(itertools.chain(*test))
        b=list(itertools.chain(*a))
        df[[name]]=pd.DataFrame(b)
    #print(df.head())
    return(df)

In [16]:
def csv2encodings(data): 
    names=data.columns.values
    lengths=[]
    encodings=[]
    known_encodings=[]
    known_names=[]
    for i in names:
        data_1=data[i]
        data_1 = data_1.dropna()
        data_2=data_1.to_list()
        encodings.append(data_2)
        lengths.append(len(data_2)/128)

    for j in range(0,len(encodings)):
        for i in range(0,round(len(encodings[j])/128) ):
            x=0
            y=128
            known_encodings.append(np.array(encodings[j][x:y]))
            x = (x+128)+1
            y= y+128#

    for i in range(0,len(lengths)):
        for j in range(0,round(lengths[i])):
            known_names.append(names[i])
    #print(lengths)
    #print(names)
    return(known_names,known_encodings)

In [17]:
from google.colab.patches import cv2_imshow

In [18]:
from google.colab import files
files.upload()

Saving encoding.csv to encoding (1).csv


{'encoding.csv': b'"Darshan,Vivek,harish,Sathish,Anindita,Abhiroop,Prashantha,Debyendu Dey,Harish,Vivek Muraleedharan,James Ajeeth"\r\n"-0.141976312,-0.120766468,-0.146423697,-0.125963748,-0.067408405,-0.15050754,-0.128416166,-0.15962742300000002,-0.088019304,-0.1017017588019371,-0.10024460405111313"\r\n"0.00258876,0.098219551,0.088945501,-0.019857496,0.016406815,0.138736129,0.059278779,0.12834017,0.047893405,0.0622192919254303,0.05932195112109184"\r\n"-0.009749264,-0.006743179,0.014782903,0.040006123,0.027709581,0.043995377,0.103477091,-0.001202059,0.06059926700000001,0.02403281070291996,0.10969705879688263"\r\n"-0.066246808,-0.075809129,-0.039824791,-0.007705198000000002,-0.029159149,0.004728511,-0.023915263,-0.066211,-0.037857156,-0.027431588619947437,0.02591830864548683"\r\n"-0.06980764099999999,-0.035339057,0.0300634,-0.072141312,-0.13396370400000002,0.008314165,-0.113412693,-0.083098255,0.030682752,-0.009559722617268562,-0.06981310248374939"\r\n"-0.016149471999999998,-0.012935214

In [19]:

def start():
    data = pd.read_csv('/content/encoding.csv')
    face_known_name,face_known_encodings_flat=csv2encodings(data)
    name = ""
    pred_names = []
    VideoCapture()
    eval_js('create()')
    try:
        while True:
            byte = eval_js('capture()')
            frame = byte2image(byte)

            # Read the frame
            #_, img = cap.read()

            # Detect the faces

            face_locations = face_recognition.face_locations(frame)
            if len(face_locations)!= 0:
                face_encoding = face_recognition.face_encodings(frame,known_face_locations=face_locations)[0] 

            else:
                pred_names = []
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame,"finding new face",(25,25) ,font, 1.0, (255, 0, 0), 1)

            #Draw the rectangle around each face

            for face_locations in face_locations:
                cv2.rectangle(frame,(face_locations[3],face_locations[0]), (face_locations[1],face_locations[2]), (255, 0, 0), 2)

                face_recognize = face_recognition.face_distance(face_known_encodings_flat,face_encoding)
                face_recognize_test = face_recognize < 0.6
                true= sum(face_recognize_test)
                if true >= 30:
                    best_match_index = np.argmin(face_recognize)
                    pred_names.append(face_known_name[best_match_index])
                    name = "identifying ...."
                    if len(pred_names) == 5:
                        name = stats.mode(pred_names)
                        name = name.mode.item()
                        engine.say("Good morning,"+name)
                        engine.say("Have a great day!")
                        engine.runAndWait()
                        engine.stop()
                        pred_names = []

                elif true < 30:
                    name = "unknown face"
                    engine.say("Sorry !")
                    engine.say("Not able to identify who you are")
                    engine.runAndWait()
                    engine.stop()

            # Display
                if name != "":
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (face_locations[3] + 6, face_locations[2]- 6), font, 0.5, (255, 255, 255), 1)
            
            eval_js('showimg("{}")'.format(image2byte(frame)))
           

    except KeyboardInterrupt:
        print("done!")
        cap.release()
        cv2.destroyAllWindows()



In [ ]:
start()